# Demonstration: point_defect_formation_static

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-06-30

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)

- - -

## Software

This notebook was tested with:

- LAMMPS (Version 2016-03-11)

- Python (Version 2.7.11)

- Jupyter (Version 1.0.0)

__NOTE__: The potential embedded in this Notebook is a MEAM potential.  Running this script with the embedded potential requires that the LAMMPS executable being used has been built with the optional meam package. 

- - -

## Introduction

This Notebook outlines the code performed by the calc_point_defect_formation_static.py calculation and offers a fully self-contained demonstration of the calculation in action.

The calculation script constructs a perfect periodic crystal, and then inserts a point defect.  Both the perfect system and defect system are relaxed with an energy/force minimization and the total potential energy is computed. The change in energy between the perfect system and the defect system is used to obtain the point defect formation energy.  

- - -

## 1. Initial Setup

1. __Necessary Python Libraries__: This is a list of the Python libraries used to run this code. 

2. __Set Working Directory__: Change the directory for performing the calculation.

3. __Input Parameter File__: An embedded copy of the input file read by the script is described.

### 1.1 Necessary Python Libraries

This is a list of the Python libraries used to run this code. 

- [numpy](http://www.numpy.org/) (Version 1.10.04)

- [atomman](https://github.com/usnistgov/atomman) (Version 0.6)

- [DataModelDict](https://github.com/usnistgov/DataModelDict) (Version 0.8)

In [1]:
import os
import sys
import random
import matplotlib.pyplot as plt
import numpy as np
import uuid
from copy import deepcopy

import iprPy
from DataModelDict import DataModelDict as DM
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

## 1.2 Set Working Directory

This allows for the working directory of the calculation to be changed.

In [2]:
#Specify working directory
working_dir = 'point_defect_formation_static'


#Check if a relative working_dir matches the current directory's name.
#This ensures that recursive directories are not created if this cell is called multiple times.
cwd = os.path.basename(os.getcwd())
if cwd != working_dir:

    #Move to working_dir and create if needed
    working_dir = os.path.abspath(working_dir)
    try:
        os.chdir(working_dir)
    except:
        os.makedirs(working_dir)
        os.chdir(working_dir)

## 1.2 Input Parameter File

This code creates a copy of the input file as read in by the calculation script. Order of the terms doesn't matter. Lines that start with #, or only have terms without values are ignored. 

__NOTE__: The potential_file '2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json' and its associated artifacts are embedded in this Notebook in Sections 2.1 and 2.2, respectively.  The load system_model file 'A1--Cu--fcc.json' is embedded in this Notebook in Section 2.3.  The ptd_model file 'vacancy/json' is embedded in this Notebook in Section 2.4. These embedded files are automatically generated when this Notebook is executed. 

The input file terms used for this calculation are:

Commands

- __lammps_command__: the path to the executable for running LAMMPS on your system.

- __mpi_command__: the command associated with calling LAMMPS to run in parallel on your system (optional).

Potential

- __potential_file__: the path to the LAMMPS-potential data model used by atomman to generate the proper LAMMPS commands for an interatomic potential.
 
- __potential_dir__: the path to the directory containing any potential artifacts (eg. eam setfl files) that are used. Default is '' (working directory for the simulation.)

System Load

- __load__: the style and path to the initial configuration file being read in. The style can be any file type supported by atomman.System.load()
 
- __load_options__: a list of key-value pairs for the optional arguments in atomman.System.load()
 
- __symbols__: a space-delimited list of elemental symbols corresponding to the atom types and potential.  Optional if the elemental information is contained in the file indicated in load.
 
- __box_parameters__: new box parameters to scale the loaded system to. Optional, although should be used if the loaded system is a prototype. 
    
    - three floats: a b c.  Allows for the definintion of orthorhombic lattice parameters.
    
    - six floats: a b c alpha beta gamma.  Allows for the definition of the lattice parameters and angles.

System Manipulation

- __x-axis, y-axis, z-axis__: optional transformation axes for rotating the system. Each vector is given by three space-delimited numbers.  The vectors must be orthogonal to each other.  If the loaded system is cubic these vectors are taken as hkl crystallographic directions and the rotated system is transformed into an orthorhombic box with dimensions given by a\*sqrt(h<sup>2</sup>+k<sup>2</sup>+l<sup>2</sup>) for each axis. 

- __shift__: optional positional shift that is applied to all atoms.  The shift is applied after axis rotation, but before size_mults supersizing.

- __shift_scale__: Boolean flag indicating if the shift is absolute (False), or should be scaled relative to the system's unit cell (True).

- __size_mults__: optional parameters for supersizing the system. This may either be a list of three or six integers.

    - Three integers: mx my mz. The absolute values indicate how many times the system is multiplied along a given box axes and the sign indicates the direction.  
    
    - Six integers: nx px ny py nz pz. This allows for multiplications in both directions for each box axis. The n terms must be less than or equal to zero, and the p terms greater than or equal to zero. 

Point Defect

- __ptd_model__: the path to the point-defect-parameters data model containing the parameters necessary for atomman to generate the point defect. A ptd_model file represents a complete concept of a point defect meaning that it contains all of the other ptd terms.  

- __ptd_type__: the type of point defect to create. Options are: 
    
    - 'v' or 'vacancy' for a vacancy
    
    - 's' or 'substitutional' for a substitutional
    
    - 'i' or 'interstitial' for an interstitial
    
    - 'd', 'db', or 'dumbbell' for a dumbbell

- __ptd_atype__: the integer atom type of the atom being added (for interstitial and dumbbell) or changed (for substitutional).  

- __ptd_pos__: the position for inserting the point defect.

- __ptd_aid__: the atom id to use for inserting the point defect (alternative to ptd_pos for vacancy, substitutional or dumbbell).

- __ptd_dumbbell_vect__: the vector associated with a dumbbell interstitial.

- __ptd_scale__: A Boolean flag indicating if the ptd_pos and ptd_dumbbell_vect are absolute (False) or should be scaled (True). The scaling is taken after the system is rotated to the axes, but before shift and size_mults are applied.  

Units

- __length_unit__: defines the unit of length for results, and input parameters if not specified. Default is 'angstrom'.

- __energy_unit__: defines the unit of energy for results, and input parameters if not specified. Default is 'eV'.

- __pressure_unit__: defines the unit of pressure for results, and input parameters if not specified. Default is 'GPa'.

- __force_unit__: defines the unit of pressure for results, and input parameters if not specified. Default is 'eV/angstrom'.

Minimization

- __energy_tolerance__: stopping energy tolerance to use for relaxing the system, i.e. LAMMPS etol. Default is 0.0.

- __force_tolerance__: stopping force tolerance to use for relaxing the system, i.e. LAMMPS ftol. Default is 1e-6 eV/A.

- __maximum_iterations__: integer maximum number of iterations of the minimizer, i.e. LAMMPS maxiter. Default is 100000.         

- __maximum_evaluations__: integer maximum number of evaluations of force/energy, i.e. LAMMPS maxeval. Default is 100000.   

In [3]:
with open('calc_point_defect_formation_static.in', 'w') as input_file:
    input_file.write("""
#Run script for calc_point_defect_formation_static.py

#Command lines for LAMMPS (and MPI).
#For lammps_command, exclude passing in a script, i.e. no "-in term" or "< term"
lammps_command              lmp_serial
mpi_command                 

#Paths to the potential data model file, and directory containing potential parameters
potential_file              2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json
potential_dir               

#Initial system configuration to load
load                        system_model A1--Cu--fcc.json
load_options                
symbols                     AlS
box_parameters              4.0502 4.0502 4.0502

#System manipulations
x-axis                      
y-axis                      
z-axis                      
atom_shift                  
size_mults                  5 5 5

#Defect data model
ptd_model                   vacancy.json

#Defect parameters (alternative to giving defect data model)
ptd_type                    
ptd_atype                   
ptd_pos                     
ptd_dumbbell_vect
ptd_aid
ptd_scale                   

#Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 

#Run parameters
strain_range                
pressure_xx                 
pressure_yy                 
pressure_zz                 
""")

## 2. Additional Data Files

To make this notebook self-contained, we have embedded a few files.

1. __MEAM Potential.__ Generates library and parameter files for one MEAM potential.

2. __Potential Data Model.__ Collects all input parameters assoiciated with running the MEAM potential in LAMMPS.

3. __Prototype Data Model.__ Generates an instance of the crystal prototype data model used to generate various structures.

4. __Point Defect Data Model.__ Collects input parameters associated with generating a point defect associated with the crystal prototype.

### 2.1 MEAM Potential

This generates the library and parameter files associated with the following interatomic potential taken from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/):

__2012--Jelinek-B--Al-Si-Mg-Cu-Fe__

__Aluminum, Silicon, Magnesium, Copper, and Iron (Al, Si, Mg, Cu, and Fe) Alloys__

*B. Jelinek, S. Groh, M. Horstemeyer, J. Houze, S.G. Kim, G.J. Wagner, A. Moitra, and M.I. Baskes, "Modified embedded atom method potential for Al, Si, Mg, Cu, and Fe alloys," Phys. Rev. B 85, 245102 (2012).*

__Notes__: This file was provided by Bohumir Jelinek (Mississippi State University) and posted on 3 July 2012. He noted, "This is a MEAM potential for Al, Si, Mg, Cu, Fe alloys. It works with LAMMPS, version 19 Jul 2011 or later, when compiled with MEAM support."

__Notice__: Users should consider all available potentials and select one which is appropriate for their use case. Use of this potential within this demonstration should not be construed as an endorsement or a recommendation. 

In [4]:
#Create MEAM library and parameter files for the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential
   
#Create MEAM library file "Jelinek_2012_meamf"
with open('Jelinek_2012_meamf', 'w') as f:
    f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meam.alsimgcufe" file providing combinational parameters
#  elt        lat     z       ielement     atwt
#  alpha      b0      b1      b2           b3    alat    esub    asub
#  t0         t1              t2           t3            rozero  ibar

'AlS'        'fcc'   12.     13           26.9815
4.64        2.04    3.00    6.0          1.50  4.05    3.353    1.07
1.0         +4.50           -2.30        8.01          1.0    -5
'SiS'        'dia'   4.      14           28.086
4.87        4.4     5.5     5.5          5.5   5.431   4.63    1.
1.0         2.05            4.47         -1.80         2.2    -5
'MgS'       'hcp'   12.      12           24.305
5.52        4.0    3.0     0.2          1.2  3.194 1.51     0.80
1.0         10.04           9.49         -4.3          0.63   -5
'CuS'        'fcc'   12.     29           63.54
5.11        3.634   2.20    6            2.20  3.62    3.54    1.07
1.0         4.91            2.49         2.95          1.10   -5
'FeS'        'bcc'   8       26           55.847
5.0270      3.500   2       1.00         1     2.851   4.28    0.5550
1          -1.6             12.5          -1.40        1.0    -5""")

#Create MEAM parameter file "Jelinek_2012_meam.alsimgcufe"
with open('Jelinek_2012_meam.alsimgcufe', 'w') as f:
    f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meamf" file providing single element parameters

  Cmin(1,1,1) = 0.8
  repuls(1,1) = 0.1
  Cmin(3,3,3) = 0.8
  Cmin(4,4,4) = 0.8
  Cmin(5,5,5) = 0.68
  repuls(5,5) = 0.3
  Cmax(5,5,5) = 1.9

  nn2(1,1)=1
  nn2(1,2)=1
  nn2(1,3)=1
  nn2(1,4)=1
  nn2(1,5)=1
  nn2(2,2)=1
  nn2(2,3)=1
  nn2(2,4)=1
  nn2(2,5)=1
  nn2(3,3)=1
  nn2(3,4)=1
  nn2(3,5)=1
  nn2(4,4)=1
  nn2(4,5)=1
  nn2(5,5)=1

  lattce(1,2)='b1'
  delta(1,2)=+0.28
  alpha(1,2)=4.56
  re(1,2)=2.62
  Cmin(1,1,2) = 0.50
  Cmin(2,2,1) = 2.00
  Cmin(1,2,1) = 2.00
  Cmin(1,2,2) = 2.00

  lattce(1,3)='b1'
  delta(1,3)=+0.23
  alpha(1,3)=4.52
  re(1,3)=2.87
  Cmin(1,1,3) = 2.00
  Cmin(3,3,1) = 0.00
  Cmin(1,3,1) = 2.00
  Cmin(1,3,3) = 0.00

  lattce(1,4)='b1'
  delta(1,4)=+0.19
  alpha(1,4)=4.65
  re(1,4)=2.53
  Cmin(1,1,4) = 0.00
  Cmin(4,4,1) = 2.00
  Cmin(1,4,1) = 2.00
  Cmin(1,4,4) = 2.00

  lattce(1,5)='b1'
  delta(1,5)=+0.26
  alpha(1,5)=4.64
  re(1,5)=2.45
  Cmin(1,1,5) = 0.90
  Cmin(5,5,1) = 0.10
  Cmin(1,5,1) = 2.00
  Cmin(1,5,5) = 2.00

  lattce(2,3)='b1'
  delta(2,3)=+0.2
  alpha(2,3)=4.73
  re(2,3)=2.75
  Cmin(2,2,3) = 1.00
  Cmin(3,3,2) = 1.00
  Cmin(2,3,2) = 2.00
  Cmin(2,3,3) = 2.00

  lattce(2,4)='b1'
  delta(2,4)=+0.14
  alpha(2,4)=4.74
  re(2,4)=2.46
  Cmin(2,2,4) = 0.00
  Cmin(4,4,2) = 0.00
  Cmin(2,4,2) = 2.00
  Cmin(2,4,4) = 2.00

  lattce(2,5)='b1'
  delta(2,5)=-0.07
  alpha(2,5)=5.17
  re(2,5)=2.39
  Cmin(2,2,5) = 1.00
  Cmin(5,5,2) = 1.00
  Cmin(2,5,2) = 2.00
  Cmin(2,5,5) = 0.00
  attrac(2,5) = 0.1
  repuls(2,5) = 0.1

  lattce(3,4)='b1'
  delta(3,4)=+0.23
  alpha(3,4)=4.70
  re(3,4)=2.63
  Cmin(3,3,4) = 2.00
  Cmin(4,4,3) = 0.00
  Cmin(3,4,3) = 2.00
  Cmin(3,4,4) = 2.00

  lattce(3,5)='b1'
  delta(3,5)=+0.6
  alpha(3,5)=4.96
  re(3,5)=2.61
  Cmin(3,3,5) = 0.65
  Cmin(5,5,3) = 0.00
  Cmin(3,5,3) = 2.00
  Cmin(3,5,5) = 2.00

  lattce(4,5)='b1'
  delta(4,5)=+0.63
  alpha(4,5)=5.21
  re(4,5)=2.42
  Cmin(5,5,4)=0.00

  attrac(5,2) = 0.1
  repuls(5,2) = 0.1

  rc = 5.0
  ialloy=1
  augt1=0
  delr=0.25658351
  emb_lin_neg=1
  bkgd_dyn=1""")

### 2.2 Potential Data Model

This generates the interatomic potential data model file associated with the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential.

In [5]:
#Creates a interatomic potential data model for using the MEAM potential in LAMMPS
with open('2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json', 'w') as f:
    f.write("""{
    "LAMMPS-potential": {
        "potential": {
            "key": "1515dd80-1984-49de-bc92-d5724059ff56", 
            "id": "2012--Jelinek-B--Al-Si-Mg-Cu-Fe"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "symbol": "AlS",
                "element": "Al", 
                "mass": 26.9815                
            }, 
            {
                "symbol": "SiS",
                "element": "Si", 
                "mass": 28.086                
            }, 
            {
                "symbol": "MgS", 
                "element": "Mg",
                "mass": 24.305                
            }, 
            {
                "symbol": "CuS", 
                "element": "Cu", 
                "mass": 63.54
            }, 
            {
                "symbol": "FeS", 
                "element": "Fe", 
                "mass": 55.847
            }
        ], 
        "pair_style": {
            "type": "meam"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Jelinek_2012_meamf"
                }, 
                {
                    "symbolsList": "True"
                }, 
                {
                    "file": "Jelinek_2012_meam.alsimgcufe"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")

### 2.3 Prototype Data Model

This generates the crystal prototype data model file associated with the fcc crystal prototype.

In [6]:
#Creates an instance of a crystal prototype data model for generating crystal structures.
with open('A1--Cu--fcc.json', 'w') as f:
    f.write("""{
    "crystal-prototype": {
        "identifier": {
            "common": "face-centered cubic", 
            "tag": "fcc", 
            "prototype": "Cu", 
            "Pearson-symbol": "cF4", 
            "Strukturbericht": "A1"
        }, 
        "space-group": {
            "number": 225, 
            "Hermann-Maguin": "F m -3 m", 
            "Schoenflies": "O^5_h", 
            "Wykoff": {
                "letter": "a", 
                "multiplicity": 4
            }
        }, 
        "atomic-system": {
            "cell": {
                "cubic": {
                    "a": {
                        "value": 1.0, 
                        "unit": "scaled"
                    }
                }
            }, 
            "atom": [
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.0, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.5, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.0, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.5, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }
            ]
        }
    }
}""")

## 2.4 Point Defect Data Model

This generates the point defect data model associated with a vacancy in an fcc unit cell.

In [7]:
with open('vacancy.json', 'w') as f:
    f.write("""{
    "point-defect-parameters": {
        "point-defect": {
            "key": "fbf7033e-9c4d-4629-a1b6-264cc6d46700",
            "id": "vacancy"
        }, 
        "system-family": "A1--Cu--fcc", 
        "atomman-defect-point-parameters": {
            "ptd_type": "v", 
            "pos": [
                0.0, 
                0.0, 
                0.0
            ], 
            "scale": true
        }
    }
}""")

# 3. LAMMPS Script Templates and Generation Functions

This section contains template versions of LAMMPS simulation scripts that are used by this calculation and the functions used to fill in the templates.

1. __LAMMPS Templates__: The embedded template file(s) which LAMMPS input scripts are generated from.

2. __Script Generation Functions__: The function(s) that fill in the templates using supplied parameters.

## 3.1 LAMMPS Templates

In [8]:
with open('min.template', 'w') as f:
    f.write("""#LAMMPS input script that performs a simple energy minimization

<atomman_system_info>

<atomman_pair_info>

variable peatom equal pe/atoms
                  
thermo_style custom step lx ly lz pxx pyy pzz pe v_peatom
thermo_modify format float %.13e

dump dumpit all custom <maximum_evaluations> atom.* id type x y z
dump_modify dumpit format "%i %i %.13e %.13e %.13e"
minimize <energy_tolerance> <force_tolerance> <maximum_iterations> <maximum_evaluations>""")

## 3.2 Script Generation Functions

In [9]:
def min_script(template_file, system_info, pair_info, etol = 0.0, ftol = 1e-6, maxiter = 100000, maxeval = 100000):
    """Create lammps script for performing a simple energy minimization."""    
    
    with open(template_file) as f:
        template = f.read()
    variable = {'atomman_system_info': system_info,
                'atomman_pair_info':   pair_info,
                'energy_tolerance': etol, 
                'force_tolerance': ftol,
                'maximum_iterations': maxiter,
                'maximum_evaluations': maxeval}
    return '\n'.join(iprPy.tools.fill_template(template, variable, '<', '>'))

## 4. Python Calculation Function(s)

In [10]:
def ptd_energy(lammps_command, system, potential, symbols, ptd, mpi_command='',
               etol=0.0, ftol=1e-6, maxiter=100000, maxeval=100000):
    
    pair_info = potential.pair_info(symbols)
        
    #write system to data file
    system_info = lmp.atom_data.dump('perfect.dat', system, 
                                     units=potential.units, 
                                     atom_style=potential.atom_style)

    #create LAMMPS input script
    min_in = min_script('min.template', system_info, pair_info, etol=etol, ftol=ftol, maxiter=maxiter, maxeval=maxeval)
    with open('perfect_min.in', 'w') as f:
        f.write(min_in)

    #run lammps
    data = lmp.run(lammps_command, 'perfect_min.in', mpi_command)

    #extract cohesive energy
    e_coh = uc.set_in_units(data.finds('peatom')[-1], 
                            lmp.style.unit(potential.units)['energy'])
    
    #Rename final dump file
    try:
        os.rename('atom.'+str(int(data.finds('Step')[-1])), 'perfect.dump')
    except:
        os.remove('perfect.dump')
        os.rename('atom.'+str(int(data.finds('Step')[-1])), 'perfect.dump')
    
    #add defect(s) to system
    system = lmp.atom_dump.load('perfect.dump')    
    for params in ptd.iterfinds('atomman-defect-point-parameters'):       
        system = am.defect.point(system, **params)
        
    #write system to data file
    system_info = lmp.atom_data.dump('ptd.dat', system, 
                                     units=potential.units, 
                                     atom_style=potential.atom_style)
    
    #create LAMMPS input script
    min_in = min_script('min.template', system_info, pair_info, etol=etol, ftol=ftol, maxiter=maxiter, maxeval=maxeval)
    with open('ptd_min.in', 'w') as f:
        f.write(min_in)

    #run lammps
    data = lmp.run(lammps_command, 'ptd_min.in', mpi_command)

    #extract per-atom energy and caclulate formation energy
    d_pot_eng = uc.set_in_units(data.finds('PotEng')[-1], 
                                lmp.style.unit(potential.units)['energy'])
    e_ptd_f = d_pot_eng - e_coh * system.natoms
    
    #rename final dump file
    try:
        os.rename('atom.'+str(int(data.finds('Step')[-1])), 'ptd.dump')
    except:
        os.remove('ptd.dump')
        os.rename('atom.'+str(int(data.finds('Step')[-1])), 'ptd.dump')
    
    os.remove('atom.0')
    
    return {'e_coh':e_coh, 'e_ptd_f': e_ptd_f}



In [11]:
def check_config(dump_file, ptd, cutoff, results_dict={}):
    #Extract the parameter sets
    params = ptd.finds('atomman-defect-point-parameters')
    
    #if there is only one set, use that set
    if len(params) == 1:
        params = params[0]
    #if there are two sets (divacancy), average the positions
    elif len(params) == 2:
        di_pos = (np.array(params[0]['pos']) + np.array(params[1]['pos'])) / 2
        params = params[0]
        params['pos'] = di_pos
    
    pos = params['pos']
    
    system = lmp.atom_dump.load(dump_file)
    
    #Compute centrosummation of atoms near defect.
    #Calculate distance of all atoms from pos
    pos_vects = system.dvect(system.atoms.view['pos'], pos) 
    
    #Identify all atoms close to pos
    pos_mags = np.linalg.norm(pos_vects, axis=1)
    index = np.where(pos_mags < cutoff)
    
    #sum up the positions of the close atoms
    results_dict['centrosummation'] = np.sum(pos_vects[index], axis=0)
    
    results_dict['has_reconfigured'] = False
    if not np.allclose(results_dict['centrosummation'], np.zeros(3), atol=1e-5):
        results_dict['has_reconfigured'] = True
        
    #Calculate shift of defect atom's position if interstitial or substitutional
    if params['ptd_type'] == 'i' or params['ptd_type'] == 's':
        new_pos = system.atoms_prop(a_id=system.natoms-1, key='pos')
        results_dict['position_shift'] = pos - new_pos
       
        if not np.allclose(results_dict['position_shift'], np.zeros(3), atol=1e-5):
            results_dict['has_reconfigured'] = True
        
    #Investigate if dumbbell vector has shifted direction 
    elif params['ptd_type'] == 'db':
        db_vect = params['db_vect'] / np.linalg.norm(params['db_vect'])
        new_pos1 = system.atoms_prop(a_id=system.natoms-1, key='pos')
        new_pos2 = system.atoms_prop(a_id=system.natoms-2, key='pos')
        new_db_vect = (new_pos1 - new_pos2) / np.linalg.norm(new_pos1 - new_pos2)
        results_dict['db_vect_shift'] = db_vect - new_db_vect
        
        if not np.allclose(results_dict['db_vect_shift'], np.zeros(3), atol=1e-5):
            results_dict['has_reconfigured'] = True
    
    return results_dict

## 5. Run Calculation and Display Results

### 5.1 Run Calculation

In [16]:
#Read in parameters from input file
with open('calc_point_defect_formation_static.in') as f:
    input_dict = iprPy.calculation_read_input('point_defect_formation_static', f)

#read in potential
potential = lmp.Potential(input_dict['potential'], input_dict['potential_dir'])        

#Unscale vector parameters relative to ucell
for params in input_dict['ptd_model'].iterfinds('atomman-defect-point-parameters'):
    if 'scale' in params and params['scale'] is True:
        if 'pos' in params:
            params['pos'] = list(input_dict['ucell'].unscale(params['pos']))
        if 'db_vect' in params:
            params['db_vect'] = list(input_dict['ucell'].unscale(params['db_vect']))
        params['scale'] = False

#Run quick_a_Cij to refine values
results_dict = ptd_energy(input_dict['lammps_command'],
                          input_dict['initial_system'], 
                          potential, 
                          input_dict['symbols'], 
                          input_dict['ptd_model'],
                          mpi_command = input_dict['mpi_command'],
                          etol =        input_dict['energy_tolerance'], 
                          ftol =        input_dict['force_tolerance'], 
                          maxiter =     input_dict['maximum_iterations'],
                          maxeval =     input_dict['maximum_evaluations'])

check_config('ptd.dump', input_dict['ptd_model'], 1.05*input_dict['ucell'].box.a, results_dict)

#Save data model of results 
results = iprPy.calculation_data_model('point_defect_formation_static', input_dict, results_dict)
with open('results.json', 'w') as f:
    results.json(fp=f, indent=4)

### 5.2 Display results.json

In [17]:
print results.json(indent=4)

{
    "calculation-point-defect-formation": {
        "calculation": {
            "id": "32b74870-8cce-4800-bd52-c5deee7d7fba", 
            "script": "calc_point_defect_formation_static", 
            "run-parameter": {
                "size-multipliers": {
                    "a": [
                        0, 
                        5
                    ], 
                    "b": [
                        0, 
                        5
                    ], 
                    "c": [
                        0, 
                        5
                    ]
                }, 
                "energy_tolerance": 0.0, 
                "force_tolerance": 9.9999999999999995e-07, 
                "maximum_iterations": 100000, 
                "maximum_evaluations": 100000
            }
        }, 
        "potential": {
            "key": "1515dd80-1984-49de-bc92-d5724059ff56", 
            "id": "2012--Jelinek-B--Al-Si-Mg-Cu-Fe"
        }, 
        "system-info": {
            "a

### 5.3 Print Results

In [18]:
ecoh = results.find('cohesive-energy')
print 'E_cohesive = %.4f %s' % (ecoh['value'], ecoh['unit'])

Ef = results.find('defect-formation-energy')
print 'E_formation = %.4f %s' % (Ef['value'], Ef['unit'])

if results.find('has_reconfigured'):
    print 'WARNING! Defect has relaxed to a different configuration!'

E_cohesive = -3.3530 eV
E_formation = 0.6702 eV
